In [507]:
import warnings
warnings.filterwarnings('ignore')
#python -W ignore file.py

In [434]:
import pytrends
import datetime 
from datetime import timedelta
from pytrends.request import TrendReq
import pandas as pd
import time
import numpy as np

def GT_1mon(kw_string,geography):
    pytrend = TrendReq(hl=geography, tz=480, timeout=(100,25), retries=2, backoff_factor=1 , requests_args={'verify':False})
    df = pd.DataFrame()
    pytrend.build_payload(kw_list=[kw_string], cat=7, timeframe='today 1-m' , geo=geography, gprop='')
    pytrend.interest_over_time().get(kw_string)
    df=df.append(pytrend.interest_over_time())
    try:
        df = df.drop("isPartial",axis=1)
    except:
        return df
    else:
        return df
def GT_7day(kw_string,geography):
    pytrend = TrendReq(hl=geography, tz=480, timeout=(100,25), retries=2, backoff_factor=1 , requests_args={'verify':False})
    
    df = pd.DataFrame()
    pytrend.build_payload(kw_list=[kw_string], cat=7, timeframe='now 7-d' , geo=geography, gprop='')
    pytrend.interest_over_time().get(kw_string)
    df = df.append(pytrend.interest_over_time())
    #df.rename(transform_date , inplace =True)
    df = df.groupby('date').apply(lambda x : x.mean())
    try:
        df = df.drop("isPartial",axis=1)
    except:
        return df
    else:
        return df

In [491]:
GT_keywords_collect_del.columns[GT_keywords_collect_del.shape[1]-1]

'metal'

In [521]:
Outputs_all = pd.DataFrame()

for i in range(0 , GT_keywords_collect_del.shape[1]-3) :
    
    GT_keywords_collect_del = pd.read_csv('GT_keywords_collect_del.csv')
    GT_keywords_collect_del.drop(['Unnamed: 0'] , axis = 1 , inplace = True)
    GT_name = GT_keywords_collect_del.columns[i]
    list1 = GT_keywords_collect_del.iloc[:,i].dropna()
    
    ###抓取過去30日資料
    sum=0
    num=0
    print(GT_keywords_collect_del.columns[i])
    data = pd.DataFrame()
    for j in list1 :
        start_time = time.time()
        df = GT_1mon(j,'')
        end_time = time.time()
        data = pd.concat([data,df],axis = 1)   

        num+=1
        sum+=round(end_time - start_time, 6)
        print("Already Done (30 days) : "+'{}'.format(round(num,0)))

    ###抓取過去過去 7日的小時資料
    sum=0
    num=0
    data1 = pd.DataFrame()
    for j in list1 :
        start_time = time.time()
        df = GT_7day(j,'')
        end_time = time.time()
        data1 = pd.concat([data1,df],axis = 1)   

        num+=1
        sum+=round(end_time - start_time, 6)
        print("Already Done (7 days) : "+'{}'.format(round(num,0)))
    
    ##處理日資料，刪除周末資料
    data = data.tail(10)
    data['weekday'] = 0
    for i in range(0 , data.shape[0]) :
        data['weekday'][i] = data.index[i].weekday()
    data = data[~data['weekday'].isin([5,6])]

    ##處理小時資料，將每日 quantile 20% 刪除，算出每日平均GT，再刪除周末資料和今日資料(可能有一筆或兩筆)
    data1.reset_index(inplace = True)
    data1['date'] = data1['date'].map(lambda x : str(x).split(' ')[0])
    data1 = data1.groupby('date').apply(lambda x : x[x > x.quantile(0.2)].mean())
    data1.index = pd.to_datetime(data1.index)
    data1['weekday'] = 0
    for i in range(0 , data1.shape[0]) :
        data1['weekday'][i] = data1.index[i].weekday()
    data1 = data1[ ~data1['weekday'].isin([5,6]) ]
    data1.drop(['weekday' , 'date'] , axis = 1 , inplace = True)
    data1 = data1[data1.index != str(datetime.date.today())]

    ##將每個關鍵字平均作為指標，把 30日和 7日重複的資料做比例的平均，來做為調整的 scale
    data = data.mean(axis = 1)
    data1 = data1.mean(axis = 1)
    date1 = str(data1.head(1).index[0])[0:10]
    date2 = str(data.tail(1).index[0])[0:10]
    diff1 = data[date1:date2].diff().dropna(axis = 0)
    diff2 = data1[date1:date2].diff().dropna(axis = 0 )
    multiply = diff1/diff2
    multiply = multiply.mean()

    ##取出過去五日GT資料
    GT_last5day = data1*multiply
    GT_last5day

    ##處理 MA3、 MA3_diff
    GT_MA3 = GT_last5day.rolling(window = 3).mean()
    GT_MA3_diff = GT_MA3.diff()
    benchmark = pd.concat([GT_last5day , GT_MA3 , GT_MA3_diff] , axis = 1 )
    benchmark.columns = ['GT_last5day','GT_MA3','GT_MA3_diff']
    benchmark = benchmark.tail(1)

    ##判斷 GT是否有過 benchmark
    Outputs = pd.DataFrame()
    if benchmark['GT_last5day'][0] >= 60 or ( benchmark['GT_MA3'][0] >= 20 and benchmark['GT_MA3_diff'][0] >= 10 ) :
        Outputs[GT_name] = 1
    else :
        Outputs[GT_name] = 0
    Outputs.index = benchmark.index + timedelta(days = 1)
    Outputs_all = Outputs_all.append(Outputs,ignore_index=True)
Outputs_all

TX


ResponseError: The request failed: Google returned a response with code 429.

In [518]:
Outputs_all.fillna(method='bfill', inplace=True)
Outputs_all = Outputs_all.head(1)
Outputs_all.index = Outputs.index
Outputs_all.to_csv('Outputs_sample(1).csv')

###抓取過去30日資料
sum=0
num=0
data = pd.DataFrame()
for i in list1 :
    start_time = time.time()
    df = GT_1mon(i,'')
    end_time = time.time()
    data = pd.concat([data,df],axis = 1)   

    num+=1
    sum+=round(end_time - start_time, 6)
    print("Already Done : "+'{}'.format(round(num,0)))
    
###抓取過去過去 7日的小時資料
sum=0
num=0
data1 = pd.DataFrame()
for i in list1 :
    start_time = time.time()
    df = GT_7day(i,'')
    end_time = time.time()
    data1 = pd.concat([data1,df],axis = 1)   

    num+=1
    sum+=round(end_time - start_time, 6)
    print("Already Done : "+'{}'.format(round(num,0)))

data.tail()

data1.tail()

##處理日資料，刪除周末資料
data = data.tail(10)
data['weekday'] = 0
for i in range(0 , data.shape[0]) :
    data['weekday'][i] = data.index[i].weekday()
data = data[~data['weekday'].isin([5,6])]

##處理小時資料，將每日 quantile 20% 刪除，算出每日平均GT，再刪除周末資料和今日資料(可能有一筆或兩筆)
data1.reset_index(inplace = True)
data1['date'] = data1['date'].map(lambda x : str(x).split(' ')[0])
data1 = data1.groupby('date').apply(lambda x : x[x > x.quantile(0.2)].mean())
data1.index = pd.to_datetime(data1.index)
data1['weekday'] = 0
for i in range(0 , data1.shape[0]) :
    data1['weekday'][i] = data1.index[i].weekday()
data1 = data1[ ~data1['weekday'].isin([5,6]) ]
data1.drop(['weekday' , 'date'] , axis = 1 , inplace = True)
data1 = data1[data1.index != str(datetime.date.today())]

##將每個關鍵字平均作為指標，把 30日和 7日重複的資料做比例的平均，來做為調整的 scale
data = data.mean(axis = 1)
data1 = data1.mean(axis = 1)
date1 = str(data1.head(1).index[0])[0:10]
date2 = str(data.tail(1).index[0])[0:10]
diff1 = data[date1:date2].diff().dropna(axis = 0)
diff2 = data1[date1:date2].diff().dropna(axis = 0 )
multiply = diff1/diff2
multiply = multiply.mean()

##取出過去五日GT資料
GT_last5day = data1*multiply
GT_last5day

##處理 MA3、 MA3_diff
GT_MA3 = GT_last5day.rolling(window = 3).mean()
GT_MA3_diff = GT_MA3.diff()
benchmark = pd.concat([GT_last5day , GT_MA3 , GT_MA3_diff] , axis = 1 )
benchmark.columns = ['GT_last5day','GT_MA3','GT_MA3_diff']
benchmark = benchmark.tail(1)

##判斷 GT是否有過 benchmark
Outputs = pd.DataFrame()
if benchmark['GT_last5day'][0] >= 60 or ( benchmark['GT_MA3'][0] >= 20 and benchmark['GT_MA3_diff'][0] >= 10 ) :
    Outputs[GT_name] = ['Volatility Signal']
else :
    Outputs[GT_name] = ['None Signal']
Outputs.index = benchmark.index + timedelta(days = 1)
Outputs

Outputs

##處理日資料，刪除周末資料
data = data.tail(10)
data['weekday'] = 0
for i in range(0 , data.shape[0]) :
    data['weekday'][i] = data.index[i].weekday()
data = data[~data['weekday'].isin([5,6])]

##處理小時資料，將每日 quantile 20% 刪除，算出每日平均GT，再刪除周末資料和今日資料(可能有一筆或兩筆)
data1.reset_index(inplace = True)
data1['date'] = data1['date'].map(lambda x : str(x).split(' ')[0])
data1 = data1.groupby('date').apply(lambda x : x[x > x.quantile(0.2)].mean())
data1.index = pd.to_datetime(data1.index)
data1['weekday'] = 0
for i in range(0 , data1.shape[0]) :
    data1['weekday'][i] = data1.index[i].weekday()
data1 = data1[ ~data1['weekday'].isin([5,6]) ]
data1.drop(['weekday' , 'date'] , axis = 1 , inplace = True)
data1 = data1[data1.index != str(datetime.date.today())]

##將每個關鍵字平均作為指標，把 30日和 7日重複的資料做比例的平均，來做為調整的 scale
data = data.mean(axis = 1)
data1 = data1.mean(axis = 1)
date1 = str(data1.head(1).index[0])[0:10]
date2 = str(data.tail(1).index[0])[0:10]
diff1 = data[date1:date2].diff().dropna(axis = 0)
diff2 = data1[date1:date2].diff().dropna(axis = 0 )
multiply = diff1/diff2
multiply = multiply.mean()

##取出過去五日GT資料
GT_last5day = data1*multiply
GT_last5day

##處理 MA3、 MA3_diff
GT_MA3 = GT_last5day.rolling(window = 3).mean()
GT_MA3_diff = GT_MA3.diff()
benchmark = pd.concat([GT_last5day , GT_MA3 , GT_MA3_diff] , axis = 1 )
benchmark.columns = ['GT_last5day','GT_MA3','GT_MA3_diff']
benchmark = benchmark.tail(1)

##判斷 GT是否有過 benchmark
Outputs = pd.DataFrame()
if benchmark['GT_last5day'][0] >= 60 or ( benchmark['GT_MA3'][0] >= 20 and benchmark['GT_MA3_diff'][0] >= 10 ) :
    Outputs[GT_name] = ['Volatility Signal']
else :
    Outputs[GT_name] = ['None Signal']
Outputs.index = benchmark.index + timedelta(days = 1)
Outputs

##處理 MA3、 MA3_diff
GT_MA3 = GT_last5day.rolling(window = 3).mean()
GT_MA3_diff = GT_MA3.diff()
benchmark = pd.concat([GT_last5day , GT_MA3 , GT_MA3_diff] , axis = 1 )
benchmark.columns = ['GT_last5day','GT_MA3','GT_MA3_diff']
benchmark = benchmark.tail(1)

##判斷 GT是否有過 benchmark
Outputs = pd.DataFrame()
if benchmark['GT_last5day'][0] >= 60 or ( benchmark['GT_MA3'][0] >= 20 and benchmark['GT_MA3_diff'][0] >= 10 ) :
    Outputs['TX'] = ['Volatility Signal']
else :
    Outputs['TX'] = ['None Signal']
Outputs.index = benchmark.index + timedelta(days = 1)
Outputs

Outputs = pd.DataFrame()
if benchmark['GT_last5day'][0] >= 60 or ( benchmark['GT_MA3'][0] >= 20 and benchmark['GT_MA3_diff'][0] >= 10 ) :
    Outputs['TX'] = ['Volatility Signal']
else :
    Outputs['TX'] = ['None Signal']
Outputs.index = benchmark.index + timedelta(days = 1)

data = data.tail(10)
data['weekday'] = 0
for i in range(0 , data.shape[0]) :
    data['weekday'][i] = data.index[i].weekday()
data = data[~data['weekday'].isin([5,6])]
data

data1.reset_index(inplace = True)
data1['date'] = data1['date'].map(lambda x : str(x).split(' ')[0])
data1 = data1.groupby('date').apply(lambda x : x[x > x.quantile(0.2)].mean())
data1.index = pd.to_datetime(data1.index)
data1['weekday'] = 0
for i in range(0 , data1.shape[0]) :
    data1['weekday'][i] = data1.index[i].weekday()
data1 = data1[ ~data1['weekday'].isin([5,6]) ]
data1.drop(['weekday' , 'date'] , axis = 1 , inplace = True)
data1 = data1[data1.index != str(datetime.date.today())]
data1

data = data.mean(axis = 1)
data1 = data1.mean(axis = 1)

data1

data = data.mean(axis = 1)
data1 = data1.mean(axis = 1)
date1 = str(data1.head(1).index[0])[0:10]
date2 = str(data.tail(1).index[0])[0:10]
diff1 = data[date1:date2].diff().dropna(axis = 0)
diff2 = data1[date1:date2].diff().dropna(axis = 0 )
multiply = diff1/diff2
multiply = multiply.mean()
multiply

GT_last5day = data1*multiply
GT_last5day

sum=0
num=0
data1 = pd.DataFrame()
for i in list1 :
    start_time = time.time()
    df = GT_7day(i,'')
    end_time = time.time()
    data1 = pd.concat([data1,df],axis = 1)   

    num+=1
    sum+=round(end_time - start_time, 6)
    print("Already Done : "+'{}'.format(round(num,0)))

##全部
data_all = pd.DataFrame()
start_time_all = time.time()
sum_all = 0

for j in range(0 , GT_keywords_collect_del.shape[1]) : 
    sum=0
    num=0
    data = pd.DataFrame()

    list1 = GT_keywords_collect_del.iloc[:,j]
    list1.dropna(inplace = True)
    
    for i in list1 :
        start_time = time.time()
        df = GT.GT_1mon(i,'')
        end_time = time.time()
        data = pd.concat([data,df],axis = 1)   
        data = pd.DataFrame(data.mean(axis = 1))
        data.columns = [columns[j]]

        num+=1
        sum+=round(end_time - start_time, 6)
        print("Already Done : "+'{}'.format(round(num,0)))
    #data.reset_index(inplace = True)
    data_all = pd.concat( [data_all , data] , axis = 1)
    print("*****This One Done*****")
    total_time = time.strftime("%H:%M:%S", time.gmtime( float(sum) ))
    print("This Takes " + total_time)
    time.sleep(3)
    
end_time_all = time.time()    
sum_all+=round(end_time_all - start_time_all, 6)
print("*****ALL DONE*****")
total_time_all = time.strftime("%H:%M:%S", time.gmtime( float(sum_all) ))
print("All Take "+total_time_all)